<a href="https://colab.research.google.com/github/lillylovecode/GenerativeAI_class/blob/main/HW06_RAG01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用rPPG相關文獻做一個系統資料庫

### 1. 建立資料夾

In [ ]:
import os
upload_dir = "uploaded_docs"
os.makedirs(upload_dir, exist_ok=True)
print(f"請將你的 .txt, .pdf, .docx 檔案放到這個資料夾中： {upload_dir}")

請將你的 .txt, .pdf, .docx 檔案放到這個資料夾中： uploaded_docs


### 2. 更新必要套件並引入

In [ ]:
!pip install -U langchain langchain-community pypdf python-docx sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings #使用HuggingFace的Embedding model
from langchain.vectorstores import FAISS #meta出的建立向量資料庫套件

### 3. 依 e5 建議加入

自訂支援 E5 的 embedding 模型（加上 "passage:" 或 "query:" 前綴）

會選擇e5系列的原因是因為支援多語言，其他的model大多只有在英文做得好。

passage 代表資料庫的`訊息`，query代表查詢的`問題`，有做前綴動作效果會好很多


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts) #繼承HuggingFace的原功能 讀取先備知識

    def embed_query(self, text):
        return super().embed_query(f"query: {text}") #繼承原功能 進行查詢

### 4. 載入文件

In [ ]:
folder_path = upload_dir
documents = []
for file in os.listdir(folder_path):
    path = os.path.join(folder_path, file)
    if file.endswith(".txt"):
        loader = TextLoader(path)
    elif file.endswith(".pdf"):
        loader = PyPDFLoader(path)
    elif file.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(path)
    else:
        continue
    documents.extend(loader.load())

### 5. 建立向量資料庫

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100) #這裡的參數可以調整chunk_size(文字塊大小) chunk_overlap(文字塊重疊) 會影響到效果
split_docs = splitter.split_documents(documents)

確認文件有放入資料夾且有切分成功

In [ ]:
print("Number of documents loaded:", len(documents))  # Check the number of loaded documents
print("Number of split documents:", len(split_docs))  # Check the number of documents after splitting


Number of documents loaded: 247
Number of split documents: 531


In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small") #這裡設定欲使用的model
vectorstore = FAISS.from_documents(split_docs, embedding_model) #開始切分文字變成向量

### 6. 儲存向量資料庫

In [ ]:
vectorstore.save_local("faiss_db") #完成後會變成資料夾

In [ ]:
!zip -r faiss_db.zip faiss_db

  adding: faiss_db/ (stored 0%)
  adding: faiss_db/index.pkl (deflated 67%)
  adding: faiss_db/index.faiss (deflated 8%)


In [ ]:
print("✅ 壓縮好的向量資料庫已儲存為 'faiss_db.zip'，請下載此檔案備份。")

✅ 壓縮好的向量資料庫已儲存為 'faiss_db.zip'，請下載此檔案備份。
